In [171]:
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Optional, Set, Any, Literal

import math

from OCC.Core.STEPControl import STEPControl_Reader, STEPControl_AsIs
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopExp import TopExp_Explorer, _TopExp, topexp
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE, TopAbs_FORWARD, TopAbs_REVERSED
from OCC.Core.TopoDS import topods_Edge, topods_Face, TopoDS_Shape, TopoDS_Face
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAbs import GeomAbs_Plane, GeomAbs_Cylinder, GeomAbs_Cone
from OCC.Core.gp import gp_Ax1, gp_Pnt, gp_Dir

from OCC.Core.Bnd import Bnd_OBB
from OCC.Core.BRepBndLib import brepbndlib_AddOBB
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.GProp import GProp_GProps


In [172]:
from OCC.Core.TopTools import (
    TopTools_IndexedDataMapOfShapeListOfShape,
    TopTools_IndexedMapOfShape,
    TopTools_ListIteratorOfListOfShape,
)

from typing import List, Dict
from OCC.Core.BRepClass3d import BRepClass3d_SolidClassifier
from OCC.Core.TopAbs import TopAbs_IN, TopAbs_OUT, TopAbs_ON

In [ ]:
# =========================
# Dataclasses: грани
# =========================

@dataclass
class PlanarFaceInfo:
    face_index: int
    point: Tuple[float, float, float]
    normal: Tuple[float, float, float]
    area: float
    orientation: str  # FORWARD / REVERSED / ...

    def __repr__(self):
        px, py, pz = self.point
        nx, ny, nz = self.normal
        return (f"PlanarFace(face_index={self.face_index}, "
                f"point=({px:.3f}, {py:.3f}, {pz:.3f}), "
                f"normal=({nx:.3f}, {ny:.3f}, {nz:.3f}), "
                f"area={self.area:.3f}, "
                f"orientation={self.orientation})")


@dataclass
class CylindricalFaceInfo:
    face_index: int

    axis_origin: Tuple[float, float, float]
    axis_dir: Tuple[float, float, float]
    radius: float

    u_first: float
    u_last: float
    u_span: float
    v_first: float
    v_last: float
    v_span: float

    is_u_closed: bool
    is_v_closed: bool

    orientation: str  # FORWARD / REVERSED / ...

    area: float
    mid_point: Tuple[float, float, float]

    topods_face: TopoDS_Face # ?

    def __repr__(self):
        ox, oy, oz = self.axis_origin
        dx, dy, dz = self.axis_dir
        mx, my, mz = self.mid_point
        return (f"CylFace(idx={self.face_index}, R={self.radius:.3f}, "
                f"axis_origin=({ox:.3f},{oy:.3f},{oz:.3f}), "
                f"axis_dir=({dx:.3f},{dy:.3f},{dz:.3f}), "
                f"u_span={self.u_span:.3f}, v_span={self.v_span:.3f}, "
                f"area={self.area:.3f}, mid=({mx:.3f},{my:.3f},{mz:.3f}), "
                f"u_closed={self.is_u_closed}, v_closed={self.is_v_closed}, "
                f"orientation={self.orientation}), "
                #f"_topods_face={self._topods_face}"
                )

@dataclass
class BasicFillet:
    cyl_face: CylindricalFaceInfo
    kind: str
    plane_plane_angle_deg: float | None
    n_neighbor_planes: int
    n_neighbor_cyls: int

# =========================
# Dataclasses: фичи (группы граней)
# =========================

@dataclass
class CylindricalFeature:
    axis_origin: Tuple[float, float, float]
    axis_dir: Tuple[float, float, float]
    radius: float
    face_indices: List[int] = field(default_factory=list)
    faces: List[CylindricalFaceInfo] = field(default_factory=list)

    def __repr__(self):
        ox, oy, oz = self.axis_origin
        dx, dy, dz = self.axis_dir
        return (f"CylFeature(R={self.radius:.3f}, "
                f"axis_origin=({ox:.3f},{oy:.3f},{oz:.3f}), "
                f"axis_dir=({dx:.3f},{dy:.3f},{dz:.3f}), "
                f"faces={self.face_indices})")


@dataclass
class CylindricalSegmentGroup:
    """
    Группа НЕПОЛНЫХ цилиндрических граней (дуги, галтели) с общей осью и радиусом.
    Пока не решаем, скругление это или что-то ещё — просто сегменты одной геометрии.
    """
    axis_origin: Tuple[float, float, float]
    axis_dir: Tuple[float, float, float]
    radius: float

    face_indices: List[int] = field(default_factory=list)
    u_spans: List[float] = field(default_factory=list)
    v_spans: List[float] = field(default_factory=list)

    def __repr__(self):
        ox, oy, oz = self.axis_origin
        dx, dy, dz = self.axis_dir
        return (f"CylSegmentGroup(R={self.radius:.3f}, "
                f"axis_origin=({ox:.3f},{oy:.3f},{oz:.3f}), "
                f"axis_dir=({dx:.3f},{dy:.3f},{dz:.3f}), "
                f"faces={self.face_indices})")

@dataclass
class FilletCandidate:
    cyl_face: CylindricalFaceInfo
    neighbor_planes: List[PlanarFaceInfo]
    neighbor_cyls: List[CylindricalFaceInfo]
    kind: str                       # 'plane-plane', 'plane-cylinder', 'cylinder-cylinder', 'unknown'
    plane_plane_angle_deg: Optional[float] = None  # если есть 2 плоскости

@dataclass
class PartAnalysis:
    faces: list          # TopoDS_Face по index
    adjacency: dict[int, set[int]]

    planar_faces: list[PlanarFaceInfo]
    full_cyl_faces: list[CylindricalFaceInfo]
    partial_cyl_faces: list[CylindricalFaceInfo]

    cyl_features: list[CylindricalFeature]
    hole_features: list[CylindricalFeature]
    boss_features: list[CylindricalFeature]

    fillet_candidates: list[FilletCandidate]
    basic_fillets: list[BasicFillet]

@dataclass
class GeometryMask:
    fillet_faces_all: Set[int]       # все цилиндрические грани, помеченные как fillet-кандидаты
    fillet_faces_basic: Set[int]     # базовые (чистые) филлеты
    non_fillet_cylindrical: Set[int] # цилиндрические грани, не относящиеся к fillet-кандидатам

HoleKind = Literal["through", "blind", "unknown"]
HoleGeometryType = Literal["circular_simple", "circular_stepped", "non_circular", "unknown"]

@dataclass
class HoleSegment:
    kind: Literal["cyl", "cone"]
    radius: float | None           # для цилиндров – радиус; для конуса – базовый радиус или None
    length: float                  # длина вдоль оси
    face_indices: List[int]

@dataclass
class ChamferInfo:
    face_index: int
    length: float            # длина по образующей, мм
    semi_angle_deg: float    # полуугол конуса (уклон)
    side: str                # 'entry' / 'exit' / 'unknown'

@dataclass
class HoleAFR:
    id: int

    geometry_type: HoleGeometryType   # circular_simple / circular_stepped / non_circular / unknown
    kind: HoleKind                    # through / blind / unknown

    axis_origin: Tuple[float, float, float] | None
    axis_dir: Tuple[float, float, float] | None

    nominal_radius: float | None      # для круговых: минимальный/основной диаметр
    segments: List[HoleSegment]       # цилиндры/конусы по оси

    side_face_indices: List[int]      # все боковые грани отверстия
    opening_faces: List[int]          # торцевые лица на поверхности детали
    bottom_faces: List[int]           # дно/дна (плоское или коническое)

    chamfers: List[ChamferInfo]       # фаски (короткие конусы)

@dataclass
class ConicalFaceInfo:
    face_index: int

    apex: Tuple[float, float, float]         # вершина конуса
    axis_dir: Tuple[float, float, float]     # направление оси
    semi_angle_deg: float                    # полуугол конуса
    ref_radius: float                        # радиус на ref-плоскости (OCCT ref)
    u_span: float
    v_span: float
    length: float                            # длина фаски по образующей (вдоль v)

In [174]:
# =========================
# Вспомогательные функции
# =========================

def load_step(shape_path: str) -> TopoDS_Shape:
    """Загрузить STEP-файл и вернуть объединённый shape."""
    reader = STEPControl_Reader()
    status = reader.ReadFile(shape_path)
    if status != IFSelect_RetDone:
        raise RuntimeError(f"Не удалось прочитать STEP-файл: статус {status}")

    # Загружаем все корневые entities
    ok = reader.TransferRoots()
    if ok == 0:
        raise RuntimeError("Не удалось перенести корневые объекты из STEP")

    shape = reader.OneShape()
    return shape

def gp_dir_to_tuple(d: gp_Dir) -> Tuple[float, float, float]:
    return (d.X(), d.Y(), d.Z())


def gp_pnt_to_tuple(p: gp_Pnt) -> Tuple[float, float, float]:
    return (p.X(), p.Y(), p.Z())

def normalize_vec(v: Tuple[float, float, float]) -> Tuple[float, float, float]:
    x, y, z = v
    n = math.sqrt(x*x + y*y + z*z)
    if n < 1e-12:
        return (0.0, 0.0, 0.0)
    return (x/n, y/n, z/n)

def almost_equal(a: float, b: float, tol: float = 1e-4) -> bool: # ???
    return abs(a - b) <= tol

def vec_dot(a, b):
    return a[0]*b[0] + a[1]*b[1] + a[2]*b[2]

def vec_norm(a):
    return math.sqrt(vec_dot(a, a))

def vec_angle(a, b):
    na = vec_norm(a)
    nb = vec_norm(b)
    if na < 1e-12 or nb < 1e-12:
        return math.pi
    dot = vec_dot(a, b) / (na * nb)
    dot = max(-1.0, min(1.0, dot))
    return math.acos(dot)

def vec_almost_equal(v1: Tuple[float, float, float], 
                     v2: Tuple[float, float, float],
                     ang_tol: float = 1e-3) -> bool: # ???
    """
    Проверка близости направлений (по углу).
    Учитываем, что ось может быть направлена в +d или -d (отверстие всё равно ось одна).
    """
    x1, y1, z1 = normalize_vec(v1)
    x2, y2, z2 = normalize_vec(v2)
    dot = x1*x2 + y1*y2 + z1*z2
    dot = max(-1.0, min(1.0, dot))
    angle = math.acos(dot)
    return angle <= ang_tol or abs(angle - math.pi) <= ang_tol

def dist_between_axes(origin1, dir1, origin2):
    # минимум расстояния между двумя прямыми: origin1 + t*dir1, origin2 + s*dir2 (упрощённо)
    import numpy as np
    p1 = np.array(origin1, dtype=float)
    d1 = np.array(dir1, dtype=float)
    p2 = np.array(origin2, dtype=float)
    d2 = np.array(dir1, dtype=float)  # NB: здесь и там одна и та же ось, но оставим общую форму

    # расстояние между параллельными прямыми
    v = p2 - p1
    cross = np.cross(d1, d2)
    n2 = np.dot(cross, cross)
    if n2 < 1e-12:
        # почти параллельны
        # расстояние = длина компоненты v, перпендикулярной d1
        proj = np.dot(v, d1) / np.dot(d1, d1)
        perp = v - proj * d1
        return float(np.linalg.norm(perp))
    else:
        # общий случай – можно не усложнять, но пусть будет
        return float(abs(np.dot(v, cross)) / math.sqrt(n2))

def indexed_map_size(m) -> int:
    """
    Унифицированный способ получить размер для
    TopTools_IndexedMapOfShape и TopTools_IndexedDataMapOfShapeListOfShape.

    В разных версиях pythonocc бывает Extent() или Size().
    """
    if hasattr(m, "Extent"):
        return m.Extent()
    if hasattr(m, "Size"):
        return m.Size()
    # на всякий случай — пробуем __len__
    try:
        return len(m)
    except TypeError:
        raise AttributeError("Indexed map has neither Extent() nor Size() nor __len__")

def collect_faces_and_map(shape: TopoDS_Shape):
    """
    Возвращает:
        faces: list[TopoDS_Face] — список граней по 0-based индексу
        face_map: TopTools_IndexedMapOfShape — карта (shape -> 1-based index)
    """
    face_map = TopTools_IndexedMapOfShape()
    topexp.MapShapes(shape, TopAbs_FACE, face_map)

    n_faces = indexed_map_size(face_map)

    faces = []
    for i in range(1, n_faces + 1):   # 1..N, т.к. OCCT 1-based
        f = topods_Face(face_map.FindKey(i))
        faces.append(f)

    return faces, face_map

def build_face_adjacency(shape, face_map):
    """
    adj: dict[int, set[int]]
        face_index (0-based) -> множество соседних face_index
    """
    n_faces = indexed_map_size(face_map)
    adj = {i: set() for i in range(n_faces)}

    # edge -> list of faces
    edge2faces = TopTools_IndexedDataMapOfShapeListOfShape()
    topexp.MapShapesAndAncestors(shape, TopAbs_EDGE, TopAbs_FACE, edge2faces)

    n_edges = indexed_map_size(edge2faces)

    for i in range(1, n_edges + 1):
        faces_list = edge2faces.FindFromIndex(i)  # TopTools_ListOfShape

        face_indices = []
        it = TopTools_ListIteratorOfListOfShape(faces_list)
        while it.More():
            shp = it.Value()
            f = topods_Face(shp)
            idx_1b = face_map.FindIndex(f)   # 1-based индекс в face_map
            if idx_1b > 0:
                face_indices.append(idx_1b - 1)  # в 0-based
            it.Next()

        # все грани, встреченные на этом ребре, считаем попарно соседями
        for a in range(len(face_indices)):
            for b in range(a + 1, len(face_indices)):
                fa = face_indices[a]
                fb = face_indices[b]
                if fa != fb:
                    adj[fa].add(fb)
                    adj[fb].add(fa)

    return adj

def angle_between_normals_deg(n1: Tuple[float, float, float],
                              n2: Tuple[float, float, float]) -> float:
    x1, y1, z1 = normalize_vec(n1)
    x2, y2, z2 = normalize_vec(n2)
    dot = x1*x2 + y1*y2 + z1*z2
    dot = max(-1.0, min(1.0, dot))
    return math.degrees(math.acos(dot))

def detect_fillet_candidates(
    partial_cyl_faces: List[CylindricalFaceInfo],
    planar_by_index: dict[int, PlanarFaceInfo],
    cyl_by_index: dict[int, CylindricalFaceInfo],
    adj: dict[int, set[int]],
    min_plane_plane_angle_deg: float = 5.0,
    max_plane_plane_angle_deg: float = 175.0,
) -> List[FilletCandidate]:
    """
    Для каждой неполной цилиндрической грани:
      - смотрим соседей по графу смежности
      - делим соседей на плоскости / цилиндры
      - определяем тип кандидата:
          * 'plane-plane' – между двумя плоскостями
          * 'plane-cylinder' – между плоскостью и цилиндром
          * 'cylinder-cylinder' – между двумя цилиндрами
          * 'unknown' – что-то сложное/неоднозначное
    """

    fillets: List[FilletCandidate] = []

    for cf in partial_cyl_faces:
        fi = cf.face_index
        neighbors = adj.get(fi, set())

        neighbor_planes: List[PlanarFaceInfo] = []
        neighbor_cyls: List[CylindricalFaceInfo] = []

        for nfi in neighbors:
            if nfi in planar_by_index:
                neighbor_planes.append(planar_by_index[nfi])
            elif nfi in cyl_by_index:
                neighbor_cyls.append(cyl_by_index[nfi])
            # остальные типы поверхностей пока игнорируем

        kind = "unknown"
        plane_plane_angle_deg = None

        # 1) Классический fillet между двумя плоскостями
        if len(neighbor_planes) >= 2:
            # возьмём первые две плоскости (при желании можно перебирать все пары)
            p1, p2 = neighbor_planes[0], neighbor_planes[1]
            ang = angle_between_normals_deg(p1.normal, p2.normal)

            # если угол не почти 0 и не почти 180 → грани действительно под углом
            if (min_plane_plane_angle_deg < ang < max_plane_plane_angle_deg):
                kind = "plane-plane"
                plane_plane_angle_deg = ang

        # 2) fillet между плоскостью и цилиндром
        elif len(neighbor_planes) == 1 and len(neighbor_cyls) >= 1:
            kind = "plane-cylinder"

        # 3) между двумя цилиндрами
        elif len(neighbor_planes) == 0 and len(neighbor_cyls) >= 2:
            kind = "cylinder-cylinder"

        fillets.append(
            FilletCandidate(
                cyl_face=cf,
                neighbor_planes=neighbor_planes,
                neighbor_cyls=neighbor_cyls,
                kind=kind,
                plane_plane_angle_deg=plane_plane_angle_deg,
            )
        )

    return fillets

def select_basic_fillets(
    fillet_candidates: List[FilletCandidate],
    max_plane_neighbors: int = 3,        # при >3 плоскостях — скорее всего сложный узел
    min_angle_deg: float = 10.0,         # для plane-plane
    max_angle_deg: float = 170.0,
):
    """
    Выбираем базовые филлеты среди кандидатов.
    Без отбора по величине радиуса — только по структуре геометрии.
    """
    basic: List[BasicFillet] = []

    for fc in fillet_candidates:
        cf = fc.cyl_face

        # 1. Тип
        if fc.kind not in ("plane-plane", "plane-cylinder"):
            continue

        # 2. Ограничение на число соседних плоскостей: слишком много → сложный узел
        n_pl = len(fc.neighbor_planes)
        n_cy = len(fc.neighbor_cyls)
        if n_pl > max_plane_neighbors:
            continue

        # 3. Для plane-plane — угол между плоскостями должен быть "разумным"
        if fc.kind == "plane-plane":
            ang = fc.plane_plane_angle_deg
            if ang is None:
                continue
            if not (min_angle_deg <= ang <= max_angle_deg):
                continue

        basic.append(
            BasicFillet(
                cyl_face=cf,
                kind=fc.kind,
                plane_plane_angle_deg=fc.plane_plane_angle_deg,
                n_neighbor_planes=n_pl,
                n_neighbor_cyls=n_cy,
            )
        )

    return basic

def build_geometry_mask(
    full_cyl_faces: list[CylindricalFaceInfo],
    partial_cyl_faces: list[CylindricalFaceInfo],
    fillet_candidates: list[FilletCandidate],
    basic_fillets: list[BasicFillet],
) -> GeometryMask:
    # все цилиндрические грани
    all_cyl_ids = {cf.face_index for cf in (full_cyl_faces + partial_cyl_faces)}

    # все кандидаты в филлеты (из detect_fillet_candidates)
    all_fillet_ids = {fc.cyl_face.face_index for fc in fillet_candidates}

    # "чистые" базовые филлеты (после select_basic_fillets)
    basic_fillet_ids = {bf.cyl_face.face_index for bf in basic_fillets}

    non_fillet_cyl = all_cyl_ids - all_fillet_ids

    return GeometryMask(
        fillet_faces_all=all_fillet_ids,
        fillet_faces_basic=basic_fillet_ids,
        non_fillet_cylindrical=non_fillet_cyl,
    )

In [ ]:
# =========================
# Извлечение граней
# =========================

def extract_planar_faces(shape: TopoDS_Shape) -> List[PlanarFaceInfo]:
    """Возвращает список PlanarFaceInfo для всех плоских граней."""
    result: List[PlanarFaceInfo] = []

    exp = TopExp_Explorer(shape, TopAbs_FACE)
    face_index = 0
    while exp.More():
        face = topods_Face(exp.Current())
        adaptor = BRepAdaptor_Surface(face, True)
        surf_type = adaptor.GetType()

        if surf_type == GeomAbs_Plane:
            plane = adaptor.Plane()
            normal = plane.Axis().Direction()
            loc = plane.Location()

            # Площадь грани
            gprops = GProp_GProps()
            brepgprop_SurfaceProperties(face, gprops)
            area = gprops.Mass()

            ori = face.Orientation()
            if ori == TopAbs_FORWARD:
                orientation = "FORWARD"
            elif ori == TopAbs_REVERSED:
                orientation = "REVERSED"
            else:
                orientation = str(ori)

            info = PlanarFaceInfo(
                face_index=face_index,
                point=gp_pnt_to_tuple(loc),
                normal=normalize_vec(gp_dir_to_tuple(normal)),
                area=area,
                orientation=orientation,
            )
            result.append(info)

        face_index += 1
        exp.Next()

    return result


def extract_cylindrical_faces(shape: TopoDS_Shape,
                              full_circle_tol_deg: float = 5.0
                              ) -> Tuple[list[CylindricalFaceInfo], list[CylindricalFaceInfo]]:
    """Возвращает (full_cyl_faces, partial_cyl_faces) без классификации на отверстия/вал/бобышку."""
    full_cyl_faces = []
    partial_cyl_faces = []

    full_circle_tol = math.radians(full_circle_tol_deg)

    exp = TopExp_Explorer(shape, TopAbs_FACE)
    face_index = 0

    while exp.More():
        face = topods_Face(exp.Current())
        adaptor = BRepAdaptor_Surface(face, True)
        surf_type = adaptor.GetType()

        if surf_type == GeomAbs_Cylinder:
            cyl = adaptor.Cylinder()
            axis: gp_Ax1 = cyl.Axis()
            radius = cyl.Radius()

            u_first = adaptor.FirstUParameter()
            u_last = adaptor.LastUParameter()
            v_first = adaptor.FirstVParameter()
            v_last = adaptor.LastVParameter()

            u_span = abs(u_last - u_first)
            v_span = abs(v_last - v_first)

            axis_origin = gp_pnt_to_tuple(axis.Location())
            axis_dir = normalize_vec(gp_dir_to_tuple(axis.Direction()))

            ori = face.Orientation()
            if ori == TopAbs_FORWARD:
                orientation = "FORWARD"
            elif ori == TopAbs_REVERSED:
                orientation = "REVERSED"
            else:
                orientation = str(ori)

            # Площадь цилиндрической грани
            gprops = GProp_GProps()
            brepgprop_SurfaceProperties(face, gprops)
            area = gprops.Mass()

            # Примерная середина поверхности
            u_mid = 0.5 * (u_first + u_last)
            v_mid = 0.5 * (v_first + v_last)
            mid_pnt = adaptor.Value(u_mid, v_mid)
            mid_point = gp_pnt_to_tuple(mid_pnt)

            info = CylindricalFaceInfo(
                face_index=face_index,
                axis_origin=axis_origin,
                axis_dir=axis_dir,
                radius=radius,
                u_first=u_first,
                u_last=u_last,
                u_span=u_span,
                v_first=v_first,
                v_last=v_last,
                v_span=v_span,
                is_u_closed=adaptor.IsUClosed(),
                is_v_closed=adaptor.IsVClosed(),
                orientation=orientation,
                area=area,
                mid_point=mid_point,
                topods_face=face,
            )

            if abs(u_span - 2 * math.pi) <= full_circle_tol:
                full_cyl_faces.append(info)
            else:
                partial_cyl_faces.append(info)

        face_index += 1
        exp.Next()

    return full_cyl_faces, partial_cyl_faces

def extract_conical_faces(shape: TopoDS_Shape) -> list[ConicalFaceInfo]:
    con_faces: list[ConicalFaceInfo] = []

    exp = TopExp_Explorer(shape, TopAbs_FACE)
    face_index = 0
    while exp.More():
        face = topods_Face(exp.Current())
        adaptor = BRepAdaptor_Surface(face, True)

        if adaptor.GetType() == GeomAbs_Cone:
            cone = adaptor.Cone()

            apex = gp_pnt_to_tuple(cone.Apex())
            axis_dir = normalize_vec(gp_dir_to_tuple(cone.Axis().Direction()))
            semi_angle = cone.SemiAngle()
            semi_angle_deg = math.degrees(semi_angle)
            ref_radius = cone.RefRadius()

            u_first = adaptor.FirstUParameter()
            u_last = adaptor.LastUParameter()
            v_first = adaptor.FirstVParameter()
            v_last = adaptor.LastVParameter()
            u_span = abs(u_last - u_first)
            v_span = abs(v_last - v_first)

            # длина фаски по образующей: просто расстояние между точками на конце и начале v
            u_mid = 0.5 * (u_first + u_last)
            p1 = adaptor.Value(u_mid, v_first)
            p2 = adaptor.Value(u_mid, v_last)

            length = math.dist(gp_pnt_to_tuple(p1), gp_pnt_to_tuple(p2))

            con_faces.append(
                ConicalFaceInfo(
                    face_index=face_index,
                    apex=apex,
                    axis_dir=axis_dir,
                    semi_angle_deg=semi_angle_deg,
                    ref_radius=ref_radius,
                    u_span=u_span,
                    v_span=v_span,
                    length=length,
                )
            )

        face_index += 1
        exp.Next()

    return con_faces

# =========================
# Группировка цилиндрических граней
# =========================

def group_cylindrical_segments(partial_faces: List[CylindricalFaceInfo],
                               axis_ang_tol: float = 1e-3,
                               axis_pos_tol: float = 1e-2,
                               radius_tol: float = 1e-3) -> List[CylindricalSegmentGroup]:
    """
    Группировка НЕПОЛНЫХ цилиндрических граней в группы по (ось, радиус).
    Никакой классификации (fillet/not fillet) здесь ещё нет.
    """
    groups: List[CylindricalSegmentGroup] = []

    for cf in partial_faces:
        assigned = False

        for grp in groups:
            if not almost_equal(grp.radius, cf.radius, tol=radius_tol):
                continue
            if not vec_almost_equal(grp.axis_dir, cf.axis_dir, ang_tol=axis_ang_tol):
                continue

            dist = dist_between_axes(grp.axis_origin, grp.axis_dir, cf.axis_origin)
            if dist > axis_pos_tol:
                continue

            # та же группа
            grp.face_indices.append(cf.face_index)
            grp.u_spans.append(cf.u_span)
            grp.v_spans.append(cf.v_span)
            assigned = True
            break

        if not assigned:
            groups.append(
                CylindricalSegmentGroup(
                    axis_origin=cf.axis_origin,
                    axis_dir=cf.axis_dir,
                    radius=cf.radius,
                    face_indices=[cf.face_index],
                    u_spans=[cf.u_span],
                    v_spans=[cf.v_span],
                )
            )

    return groups

def group_full_cylinders_to_features(full_faces: List[CylindricalFaceInfo],
                                     axis_ang_tol: float = 1e-3,
                                     axis_pos_tol: float = 1e-2,
                                     radius_tol: float = 1e-3) -> List[CylindricalFeature]:
    """
    Группировка ПОЛНЫХ цилиндрических граней:
      - один CylindricalFeature ≈ одно отверстие/вал/бобышка
      - критерий: общий радиус + почти совпадающая ось
      - три отверстия в одной плоскости с параллельными осями → останутся тремя разными фичами
    """
    features: List[CylindricalFeature] = []

    for cf in full_faces:
        assigned = False

        for feat in features:
            # радиус
            if not almost_equal(feat.radius, cf.radius, tol=radius_tol):
                continue

            # направление оси
            if not vec_almost_equal(feat.axis_dir, cf.axis_dir, ang_tol=axis_ang_tol):
                continue

            # расстояние между осями
            dist = dist_between_axes(feat.axis_origin, feat.axis_dir, cf.axis_origin)
            if dist > axis_pos_tol:
                continue

            # если все три условия прошли — считаем это той же фичей
            feat.face_indices.append(cf.face_index)
            feat.faces.append(cf)
            assigned = True
            break

        if not assigned:
            features.append(
                CylindricalFeature(
                    axis_origin=cf.axis_origin,
                    axis_dir=cf.axis_dir,
                    radius=cf.radius,
                    face_indices=[cf.face_index],
                    faces=[cf],
                )
            )

    return features

def split_cyl_features_into_holes_and_bosses(
    features: List[CylindricalFeature]
):
    """
    Делим цилиндрические фичи на:
      - holes: внутренняя поверхность (отверстия)
      - bosses: внешняя поверхность (бобышки/наружные цилиндры)
      - ambiguous: спорные случаи

    Критерий пока максимально простой:
      - считаем ориентации граней внутри фичи
      - если REVERSED > FORWARD → hole
      - если FORWARD > REVERSED → boss
      - иначе → ambiguous
    """
    holes: List[CylindricalFeature] = []
    bosses: List[CylindricalFeature] = []
    ambiguous: List[CylindricalFeature] = []

    for feat in features:
        n_fwd = sum(1 for f in feat.faces if f.orientation == "FORWARD")
        n_rev = sum(1 for f in feat.faces if f.orientation == "REVERSED")

        if n_rev > n_fwd:
            holes.append(feat)
        elif n_fwd > n_rev:
            bosses.append(feat)
        else:
            ambiguous.append(feat)

    return holes, bosses, ambiguous

# =========================
# High-level: извлечение фичей из shape
# =========================
def extract_features_from_shape(shape: TopoDS_Shape,
                                full_circle_tol_deg: float = 5.0,
                                axis_ang_tol: float = 1e-3,
                                axis_pos_tol: float = 1e-2,
                                radius_tol: float = 1e-3):
    """
    High-level точка входа.

    Возвращает:
      planar_faces
      full_cyl_faces
      partial_cyl_faces
      cyl_features
      cyl_segment_groups
      hole_features
      boss_features
      ambiguous_features
    """
    planar_faces = extract_planar_faces(shape)
    full_cyl_faces, partial_cyl_faces = extract_cylindrical_faces(
        shape,
        full_circle_tol_deg=full_circle_tol_deg
    )

    cyl_features = group_full_cylinders_to_features(
        full_cyl_faces,
        axis_ang_tol=axis_ang_tol,
        axis_pos_tol=axis_pos_tol,
        radius_tol=radius_tol,
    )

    cyl_segment_groups = group_cylindrical_segments(
        partial_cyl_faces,
        axis_ang_tol=axis_ang_tol,
        axis_pos_tol=axis_pos_tol,
        radius_tol=radius_tol,
    )

    hole_features, boss_features, ambiguous_features = \
        split_cyl_features_into_holes_and_bosses(cyl_features)

    return (
        planar_faces,
        full_cyl_faces,
        partial_cyl_faces,
        cyl_features,
        cyl_segment_groups,
        hole_features,
        boss_features,
        ambiguous_features,
    )

In [ ]:
def cluster_hole_features_by_axis(
    hole_features: List["CylindricalFeature"],
    ang_tol_rad: float = math.radians(1.0),
    axis_dist_tol: float = 0.5,
):
    """
    Объединяем CylindricalFeature в кластеры по общей оси.
    Кластер = "одно физическое отверстие" (возможно ступенчатое).
    """
    clusters: List[List["CylindricalFeature"]] = []

    for feat in hole_features:
        added = False
        for cluster in clusters:
            ref = cluster[0]
            # проверяем угол между осями
            ang = vec_angle(feat.axis_dir, ref.axis_dir)
            if ang > ang_tol_rad and abs(ang - math.pi) > ang_tol_rad:
                continue
            # проверяем расстояние между осями
            d = dist_between_axes(feat.axis_origin, feat.axis_dir,
                                  ref.axis_origin)
            if d > axis_dist_tol:
                continue

            cluster.append(feat)
            added = True
            break

        if not added:
            clusters.append([feat])

    return clusters

def get_part_max_dim(shape) -> float:
    obb = Bnd_OBB()
    brepbndlib_AddOBB(shape, obb, True, True, True)
    hx = obb.XHSize()
    hy = obb.YHSize()
    hz = obb.ZHSize()
    m = 2.0 * max(hx, hy, hz)
    return m if m > 1e-6 else 100.0

def classify_hole_through_blind(
    shape,
    axis_origin: Tuple[float, float, float],
    axis_dir: Tuple[float, float, float],
) -> HoleKind:
    L = get_part_max_dim(shape) * 1.2  # немного больше детали
    ox, oy, oz = axis_origin
    dx, dy, dz = axis_dir

    p_plus = gp_Pnt(ox + dx*L, oy + dy*L, oz + dz*L)
    p_minus = gp_Pnt(ox - dx*L, oy - dy*L, oz - dz*L)

    classifier = BRepClass3d_SolidClassifier(shape)
    classifier.Perform(p_plus, 1e-3)
    st_plus = classifier.State()
    classifier.Perform(p_minus, 1e-3)
    st_minus = classifier.State()

    plus_out = (st_plus in (TopAbs_OUT, TopAbs_ON))
    minus_out = (st_minus in (TopAbs_OUT, TopAbs_ON))
    plus_in = (st_plus == TopAbs_IN)
    minus_in = (st_minus == TopAbs_IN)

    if plus_out and minus_out:
        return "through"
    if (plus_in and minus_out) or (minus_in and plus_out):
        return "blind"
    return "unknown"

def detect_holes_afr(
    shape: TopoDS_Shape,
    hole_features: List["CylindricalFeature"],
    #boss_features: List["CylindricalFeature"],
    #ambiguous_features: List["CylindricalFeature"],
    planar_faces: List["PlanarFaceInfo"],
    adj: Dict[int, set[int]],
    conical_faces: List[ConicalFaceInfo],
    chamfer_max_length: float = 2.0,
) -> List[HoleAFR]:
    # индексы -> геом. сущности
    planar_by_index = {pf.face_index: pf for pf in planar_faces}
    conical_by_index = {cf.face_index: cf for cf in conical_faces}

    # кластеризуем отверстия по оси
    #all_features = hole_features + boss_features + ambiguous_features
    #is_hole_feature = {id(f): (f in hole_features) for f in all_features}
    clusters = cluster_hole_features_by_axis(hole_features)

    holes_afr: List[HoleAFR] = []

    for hid, cluster in enumerate(clusters):
        # ось / радиус
        axis_dir = cluster[0].axis_dir
        # усреднённая точка на оси
        ax = sum(f.axis_origin[0] for f in cluster) / len(cluster)
        ay = sum(f.axis_origin[1] for f in cluster) / len(cluster)
        az = sum(f.axis_origin[2] for f in cluster) / len(cluster)
        axis_origin = (ax, ay, az)

        # все цилиндрические грани
        cyl_faces = [cf for feat in cluster for cf in feat.faces]
        cyl_face_indices = [cf.face_index for cf in cyl_faces]

        # разные радиусы в кластере
        radii = sorted({round(cf.radius, 4) for cf in cyl_faces})
        if len(radii) == 1:
            geometry_type: HoleGeometryType = "circular_simple"
            nominal_radius = radii[0]
        else:
            geometry_type = "circular_stepped"
            nominal_radius = max(radii)

        # тип отверстия (глухое/сквозное)
        hole_kind = classify_hole_through_blind(shape, axis_origin, axis_dir)

        # соседи всех граней кластера
        neighbor_indices = set()
        for fi in cyl_face_indices:
            neighbor_indices.update(adj.get(fi, set()))

        opening_faces: List[int] = []
        bottom_faces: List[int] = []
        chamfers: List[ChamferInfo] = []
        segments: List[HoleSegment] = []

        # 1) Фаски и конусное дно / countersink:
        for nfi in neighbor_indices:
            if nfi in conical_by_index:
                cone = conical_by_index[nfi]
                # соосен ли конус оси отверстия?
                dot = abs(vec_dot(cone.axis_dir, axis_dir) / (vec_norm(cone.axis_dir) * vec_norm(axis_dir)))
                if dot < 0.95:
                    continue
                # короткий конус – фаска
                if cone.length <= chamfer_max_length:
                    chamfers.append(
                        ChamferInfo(
                            face_index=nfi,
                            length=cone.length,
                            semi_angle_deg=cone.semi_angle_deg,
                            side="unknown",  # можно потом улучшить
                        )
                    )
                else:
                    # более длинный конус — сегмент профиля отверстия (коническое дно / countersink)
                    segments.append(
                        HoleSegment(
                            kind="cone",
                            radius=cone.ref_radius,
                            length=cone.length,
                            face_indices=[nfi],
                        )
                    )
                    bottom_faces.append(nfi)
        # эвристика
        if bottom_faces and hole_kind == "through":
            hole_kind = "blind"

        # 2) Плоские торцы (дно, вход/выход)
        for nfi in neighbor_indices:
            if nfi in planar_by_index:
                pf = planar_by_index[nfi]
                ndot = abs(vec_dot(pf.normal, axis_dir) / (vec_norm(pf.normal)*vec_norm(axis_dir)))
                if ndot > 0.95:
                    # почти перпендикуляр к оси → торец
                    opening_faces.append(nfi)
                    # можем при желании часть из них потом классифицировать как дно, если дырка blind

        # 3) Цилиндрические сегменты профиля
        # (Можно сгруппировать по радиусу и диапазону вдоль оси, но для начала просто один сегмент на радиус)
        for r in radii:
            faces_r = [cf for cf in cyl_faces if abs(cf.radius - r) < 1e-4]
            if not faces_r:
                continue
            # оценим длину вдоль оси как max проекции центров граней на ось
            v_min = min(cf.v_first for cf in faces_r)
            v_max = max(cf.v_last  for cf in faces_r)
            length = abs(v_max - v_min)

            segments.append(
                HoleSegment(
                    kind="cyl",
                    radius=r,
                    length=abs(length),
                    face_indices=[cf.face_index for cf in faces_r],
                )
            )

        hole = HoleAFR(
            id=hid,
            geometry_type=geometry_type,
            kind=hole_kind,
            axis_origin=axis_origin,
            axis_dir=axis_dir,
            nominal_radius=nominal_radius,
            segments=segments,
            side_face_indices=cyl_face_indices,
            opening_faces=opening_faces,
            bottom_faces=bottom_faces,
            chamfers=chamfers,
        )
        holes_afr.append(hole)

    return holes_afr

In [225]:
def analyze_shape_for_fillets(shape: TopoDS_Shape):
    # 1. Топология: список граней и граф смежности
    faces, face_map = collect_faces_and_map(shape)
    adj = build_face_adjacency(shape, face_map)

    # 2. Геометрия и фичи – через наш общий pipeline
    (planar_faces,
     full_cyl_faces,
     partial_cyl_faces,
     cyl_features,
     cyl_segment_groups,
     hole_features,
     boss_features,
     ambiguous_features) = extract_features_from_shape(shape)

    # 3. Быстрый доступ: face_index -> Planar/Cyl
    planar_by_index = {pf.face_index: pf for pf in planar_faces}
    cyl_by_index = {cf.face_index: cf for cf in (full_cyl_faces + partial_cyl_faces)}

    # 4. Кандидаты в филлеты
    fillet_candidates = detect_fillet_candidates(
        partial_cyl_faces=partial_cyl_faces,
        planar_by_index=planar_by_index,
        cyl_by_index=cyl_by_index,
        adj=adj,
    )

    # 5. Базовые (очищенные) филлеты
    basic_fillets = select_basic_fillets(fillet_candidates)

    # 6. Маска по геометрии
    geom_mask = build_geometry_mask(
        full_cyl_faces=full_cyl_faces,
        partial_cyl_faces=partial_cyl_faces,
        fillet_candidates=fillet_candidates,
        basic_fillets=basic_fillets,
    )

    # 7. Возвращаем всё, чем реально будем пользоваться
    return (planar_faces,
            full_cyl_faces,
            partial_cyl_faces,
            fillet_candidates,
            basic_fillets,
            geom_mask,
            adj)

In [226]:
# =========================
# Пример использования
# =========================

def main():
    import argparse

    parser = argparse.ArgumentParser(description="AFR v2: плоскости + цилиндры (грани и фичи).")
    parser.add_argument("step_file", help="Путь к STEP-файлу (.stp/.step)")
    args = parser.parse_args()

    print(f"Загружаю STEP: {args.step_file}")
    shape = load_step(args.step_file)

    (planar_faces,
    full_cyl_faces,
    partial_cyl_faces,
    cyl_features,
    cyl_segment_groups,
    hole_features,
    boss_features,
    ambiguous_features) = extract_features_from_shape(shape)

    # быстрые словари
    planar_by_index = {pf.face_index: pf for pf in planar_faces}
    cyl_by_index_full = {cf.face_index: cf for cf in full_cyl_faces}
    cyl_by_index_partial = {cf.face_index: cf for cf in partial_cyl_faces}

    # если хочешь общую карту цилиндров:
    cyl_by_index = {**cyl_by_index_full, **cyl_by_index_partial}

    # HOLES:
    faces, face_map = collect_faces_and_map(shape)
    adj = build_face_adjacency(shape, face_map)
    holes_afr = detect_holes_afr(shape, hole_features, planar_faces, adj)
    for h in holes_afr:
        print(f"Hole #{h.id}: kind={h.kind}, R={h.nominal_radius}, chamfers={len(h.chamfers)}")
        for ch in h.chamfers:
            print(f"   chamfer face={ch.face_index}, L={ch.length:.3f}, angle={ch.semi_angle_deg:.1f}°")

    # FIRST
    print("\nПлоские грани (первые 15):")
    for pf in planar_faces[:15]:
        print("  ", pf)

    print("\nПолные цилиндрические грани (full_cyl_faces):")
    for cf in full_cyl_faces:
        print("  ", cf)

    print("\nНеполные цилиндрические грани (partial_cyl_faces):")
    for cf in partial_cyl_faces:
        print("  ", cf)

    print("\nГруппы полных цилиндров (CylindricalFeature):")
    for feat in cyl_features:
        print("  ", feat)

    print("\nГруппы сегментов цилиндров (CylindricalSegmentGroup):")
    for grp in cyl_segment_groups:
        print("  ", grp)
    
    print("\nОтверстия (hole_features):")
    for h in hole_features:
        print("  ", h)

    print("\nНаружные цилиндры (boss_features):")
    for b in boss_features:
        print("  ", b)

    print("\nНеоднозначные фичи (ambiguous_features):")
    for a in ambiguous_features:
        print("  ", a)

    (planar_faces,
    full_cyl_faces,
    partial_cyl_faces,
    fillets,
    basic_fillets,
    geom_mask,
    adj) = analyze_shape_for_fillets(shape)


    print("\nНайденные fillet-кандидаты:")
    for fc in fillets:
        cf = fc.cyl_face
        print(f"- CylFace idx={cf.face_index}, R={cf.radius}, kind={fc.kind}, "
            f"neighbors: planes={len(fc.neighbor_planes)}, cyls={len(fc.neighbor_cyls)}, "
            f"angle={fc.plane_plane_angle_deg}")
    
    basic_fillets = select_basic_fillets(fillets)
    for bf in basic_fillets:
        cf = bf.cyl_face
        print(f"idx={cf.face_index}  R={cf.radius}  planes={bf.n_neighbor_planes}  kind={bf.kind}")
    
    geometry_mask = build_geometry_mask(full_cyl_faces, partial_cyl_faces, fillets, basic_fillets) 
    
    all_ids   = {fc.cyl_face.face_index for fc in fillets}
    basic_ids = {bf.cyl_face.face_index for bf in basic_fillets}

    pure_fillet_like = basic_ids                # простые понятные галтели
    complex_curved   = all_ids - basic_ids      # сложные криволинейные места


In [227]:
def analyze_part(shape: TopoDS_Shape) -> PartAnalysis:
    faces, face_map = collect_faces_and_map(shape)
    adj = build_face_adjacency(shape, face_map)

    (planar_faces,
    full_cyl_faces,
    partial_cyl_faces,
    cyl_features,
    cyl_segment_groups,
    hole_features,
    boss_features,
    ambiguous_features) = extract_features_from_shape(shape)
    
    hole_features, boss_features, ambiguous = split_cyl_features_into_holes_and_bosses(cyl_features)

    fillet_candidates = detect_fillet_candidates(
        partial_cyl_faces,
        planar_by_index={pf.face_index: pf for pf in planar_faces},
        cyl_by_index={cf.face_index: cf for cf in full_cyl_faces + partial_cyl_faces},
        adj=adj,
    )
    basic_fillets = select_basic_fillets(fillet_candidates)

    return PartAnalysis(
        faces=faces,
        adjacency=adj,
        planar_faces=planar_faces,
        full_cyl_faces=full_cyl_faces,
        partial_cyl_faces=partial_cyl_faces,
        cyl_features=cyl_features,
        hole_features=hole_features,
        boss_features=boss_features,
        fillet_candidates=fillet_candidates,
        basic_fillets=basic_fillets,
    )

In [ ]:
shape = load_step("../data/example_complex.stp")

In [229]:
(planar_faces,
    full_cyl_faces,
    partial_cyl_faces,
    cyl_features,
    cyl_segment_groups,
    hole_features,
    boss_features,
    ambiguous_features) = extract_features_from_shape(shape)


In [230]:
part_analysis = analyze_part(shape)

In [ ]:
(planar_faces,
    full_cyl_faces,
    partial_cyl_faces,
    cyl_features,
    cyl_segment_groups,
    hole_features,
    boss_features,
    ambiguous_features) = extract_features_from_shape(shape)
for cf in full_cyl_faces:
        print("  ", cf)

   CylFace(idx=53, R=20.000, axis_origin=(15478.719,-1609.992,1194.896), axis_dir=(-0.478,0.222,0.850), u_span=6.283, v_span=2.500, area=314.159, mid=(15456.149,-1607.601,1194.224), u_closed=True, v_closed=False, orientation=REVERSED), 
   CylFace(idx=54, R=20.000, axis_origin=(15414.542,-1580.135,1150.978), axis_dir=(-0.478,0.222,0.850), u_span=6.283, v_span=2.500, area=314.159, mid=(15391.972,-1577.745,1150.306), u_closed=True, v_closed=False, orientation=REVERSED), 
   CylFace(idx=55, R=20.000, axis_origin=(15346.513,-1548.487,1104.423), axis_dir=(-0.478,0.222,0.850), u_span=6.283, v_span=27.500, area=3455.752, mid=(15317.968,-1543.316,1114.373), u_closed=True, v_closed=False, orientation=REVERSED), 
   CylFace(idx=139, R=22.500, axis_origin=(15328.826,-1540.259,1135.863), axis_dir=(0.478,-0.222,-0.850), u_span=6.283, v_span=25.000, area=3534.292, mid=(15315.191,-1543.038,1114.210), u_closed=True, v_closed=False, orientation=FORWARD), 
   CylFace(idx=151, R=22.500, axis_origin=(1539

In [232]:
faces, face_map = collect_faces_and_map(shape)
print("faces:", len(faces))

adj = build_face_adjacency(shape, face_map)
print("adj len:", len(adj))

for i in range(5):
    print(f"face {i} -> neighbors:", sorted(adj[i]))

faces: 157
adj len: 157
face 0 -> neighbors: [25, 95, 107, 110, 124]
face 1 -> neighbors: [6, 11, 112, 121]
face 2 -> neighbors: [6, 67, 108, 109, 129]
face 3 -> neighbors: [19, 24, 72, 95, 102, 106, 132]
face 4 -> neighbors: [58, 123, 124, 125, 126, 127, 128, 129, 130]


In [234]:
conical_faces = extract_conical_faces(shape)

holes_afr = detect_holes_afr(
    shape,
    hole_features,
    planar_faces,
    adj,
    conical_faces,
    chamfer_max_length=2.0,
)

for h in holes_afr:
    print(f"Hole #{h.id}: geom={h.geometry_type}, kind={h.kind}, Rnom={h.nominal_radius}")
    print(f"  segments:")
    for s in h.segments:
        print(f"    {s.kind} R={s.radius} L={s.length:.2f}, faces={s.face_indices}")
    print(f"  chamfers: {[ (c.face_index, round(c.length,3), round(c.semi_angle_deg,1)) for c in h.chamfers ]}")
    print(f"  opening_faces={h.opening_faces}, bottom_faces={h.bottom_faces}")

Hole #0: geom=circular_simple, kind=through, Rnom=20.0
  segments:
    cyl R=20.0 L=2.50, faces=[53]
  chamfers: [(62, 1.414, 45.0)]
  opening_faces=[56], bottom_faces=[]
Hole #1: geom=circular_stepped, kind=through, Rnom=5.0
  segments:
    cone R=5.0 L=5.83, faces=[155]
    cyl R=5.0 L=10.00, faces=[156]
    cyl R=12.5 L=10.00, faces=[154]
    cyl R=20.0 L=2.50, faces=[54]
  chamfers: [(61, 1.414, 45.0)]
  opening_faces=[152, 153], bottom_faces=[155]
Hole #2: geom=circular_simple, kind=through, Rnom=20.0
  segments:
    cyl R=20.0 L=27.50, faces=[55]
  chamfers: [(60, 1.414, 45.0)]
  opening_faces=[140], bottom_faces=[]


In [208]:
def debug_face_neighbors(
    face_index: int,
    planar_by_index: dict[int, PlanarFaceInfo],
    cyl_by_index: dict[int, CylindricalFaceInfo],
    adj: dict[int, set[int]],
):
    print(f"\nFace {face_index}:")
    if face_index in planar_by_index:
        print("  Type: PLANAR")
        print("   ", planar_by_index[face_index])
    elif face_index in cyl_by_index:
        print("  Type: CYL")
        print("   ", cyl_by_index[face_index])
    else:
        print("  Type: OTHER")

    for ni in sorted(adj.get(face_index, [])):
        if ni in planar_by_index:
            t = "PLANAR"
        elif ni in cyl_by_index:
            t = "CYL"
        else:
            t = "OTHER"
        print(f"    neighbor {ni}: {t}")


In [209]:
planar_by_index = {pf.face_index: pf for pf in planar_faces}
cyl_by_index = {cf.face_index: cf for cf in (full_cyl_faces + partial_cyl_faces)}

debug_face_neighbors(39, planar_by_index, cyl_by_index, adj)


Face 39:
  Type: CYL
    CylFace(idx=39, R=6.000, axis_origin=(15279.923,-1558.971,1220.000), axis_dir=(0.000,0.000,-1.000), u_span=1.203, v_span=23.986, area=154.607, mid=(15282.976,-1564.137,1097.668), u_closed=False, v_closed=False, orientation=REVERSED), 
    neighbor 27: CYL
    neighbor 37: PLANAR
    neighbor 41: OTHER
    neighbor 79: OTHER
